## Unsupervised Learning Strategy

* Download/load SP500 stocks price data
* Aggregate on monthly level and filter top 150 most liquid stocks
* Calculate monthly returns for differnt time-horizons
* Download Fama-French Factors and calculate rolling factor betas
* For each month fit a k-means clustering algorithm to group similar assets based on their features
* For each month select assets based on the cluster and form a portfolio based on efficent frontier max sharpe ratio optimization
* Visualize portfolio returns and compare to Sp500 returns

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.regression.rolling import RollingOLS
import yfinance as yf
import pandas_datareader.data as web
import statsmodels.api as sm
import datetime as dt
import pandas_ta
import warnings
warnings.filterwarnings("ignore")

**Download SP500 stock data**

In [18]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols = list(sp500['Symbol'].unique())
symbols

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'BXP',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CHRW',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG'

In [44]:
end_date = '2023-09-27'
start_date = pd.to_datetime(end_date) - pd.DateOffset(years=5)

df = yf.download(tickers=symbols,
                 start=start_date, 
                 end=end_date).stack()

df

[*********************100%%**********************]  503 of 503 completedmpleted

4 Failed downloads:
[*********************100%%**********************]  503 of 503 completed['SOLV', 'SW', 'GEV', 'VLTO']: YFChartError("%ticker%: Data doesn't exist for startDate = 1538020800, endDate = 1695787200")
[***************************127%%*****************************]  639 of 503 completed

Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2018-09-27 A        67.763641   70.800003   70.849998   70.099998   70.580002   
           AAL      40.727211   41.500000   42.200001   41.150002   41.230000   
           AAPL     53.586349   56.237499   56.610001   55.884998   55.955002   
           ABBV     71.712509   94.139999   94.889999   93.959999   94.349998   
           ABT      65.870644   73.019997   73.180000   72.690002   73.019997   
...                       ...         ...         ...         ...         ...   
2023-09-26 XEL      56.057709   57.720001   59.480000   57.410000   59.480000   
           XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211006  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZTS     175.131119  176.869995  178.449997  176.270004  176.580002   

Price                   Volume  
Date       Ticker               
2018-09-27 A         1581700.0  
           AAL       5654600.0  
           AAPL    120724800.0  
           ABBV      3028600.0  
           ABT       5493900.0  
...                        ...  
2023-09-26 XEL       6275500.0  
           XYL       1322400.0  
           YUM       1500600.0  
           ZBH       3610500.0  
           ZTS       1463200.0  

[454010 rows x 6 columns]

In [46]:
df.columns = df.columns.str.lower()
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2018-09-27 A        67.763641   70.800003   70.849998   70.099998   70.580002   
           AAL      40.727211   41.500000   42.200001   41.150002   41.230000   
           AAPL     53.586349   56.237499   56.610001   55.884998   55.955002   
           ABBV     71.712509   94.139999   94.889999   93.959999   94.349998   
           ABT      65.870644   73.019997   73.180000   72.690002   73.019997   
...                       ...         ...         ...         ...         ...   
2023-09-26 XEL      56.057709   57.720001   59.480000   57.410000   59.480000   
           XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211006  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZTS     175.131119  176.869995  178.449997  176.270004  176.580002   

Price                   volume  
Date       Ticker               
2018-09-27 A         1581700.0  
           AAL       5654600.0  
           AAPL    120724800.0  
           ABBV      3028600.0  
           ABT       5493900.0  
...                        ...  
2023-09-26 XEL       6275500.0  
           XYL       1322400.0  
           YUM       1500600.0  
           ZBH       3610500.0  
           ZTS       1463200.0  

[454010 rows x 6 columns]